In [1]:
import torch
#GPU使用可能確認
print(torch.cuda.is_available())
#使用できるデバイス数確認
print(torch.cuda.device_count())

True
2


In [2]:
!nvidia-smi

Mon Jul 31 11:50:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|


|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 35%   45C    P8    39W / 350W |    308MiB / 24576MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:C1:00.0 Off |                  N/A |
| 30%   42C    P8    19W / 350W |     13MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                        

In [3]:
! pip install transformers
! pip install mojimoji
! pip install pyknp

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import random
import re
import string
import time
from tqdm import tqdm
import mojimoji
import pandas as pd
from transformers import BertConfig, BertModel, BertTokenizer
from pyknp import Juman

###　ターミナルで形態素解析juman++のインストール
https://qiita.com/Gushi_maru/items/ee434b5bc9f020c8feb6
・jumanpp-2.0.0-rc3 download
wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz
・unzip a file
tar xvf jumanpp-2.0.0-rc3.tar.xz
・コンパイラのインストール
sudo apt update -y
sudo apt upgrade -y
sudo apt install build-essential -y
・cmakeインストール
sudo apt install cmake -y
・コンパイラ用フォルダ作成
cd [jumanディレクトリのパスの指定]
mkdir build
・juman++コンパイル
cd [buildのパス指定]
cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
make
・コンパイル完了したらインストール開始
sudo make install


In [5]:
#juman++動作確認
! echo すももももももももももも | jumanpp

すもも すもも すもも 名詞 6 普通名詞 1 * 0 * 0 "自動獲得:テキスト"
@ すもも すもも すもも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:酸桃/すもも 自動獲得:EN_Wiktionary"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
も も も 助詞 9 副助詞 2 * 0 * 0 NIL
もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:桃/もも ドメイン:料理・食事 カテゴリ:植物;人工物-食べ物 漢字読み:訓"
@ もも もも もも 名詞 6 普通名詞 1 * 0 * 0 "代表表記:股/もも カテゴリ:動物-部位"
EOS


### jiuman++のバージョン確認

In [6]:
! jumanpp -v

Juman++ Version: 2.0.0-rc3 / Dictionary: 20190731-356e143 / LM: K:20190430-7d143fb L:20181122-b409be68 F:20171214-9d125cb


In [7]:
# JUMANの動作確認
from pyknp import Juman
text = "自然言語処理のBERTについて学習中です。"
juman = Juman()
result =juman.analysis(text)
result = [mrph.midasi for mrph in result.mrph_list()]
print(text)
print(result)

自然言語処理のBERTについて学習中です。
['自然', '言語', '処理', 'の', 'BERT', 'に', 'ついて', '学習', '中', 'です', '。']


### ダウンロードしたモデルのロード

In [8]:
"""
import torch
vocab_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/vocab.txt"
config_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/config.json"
model_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/pytorch_model.bin"
"""

'\nimport torch\nvocab_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/vocab.txt"\nconfig_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/config.json"\nmodel_file = "/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/deberta_models/pytorch_model.bin"\n'

### 京都大学が公開している日本語Debertaモデル
https://huggingface.co/ku-nlp/deberta-v2-base-japanese

In [63]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("ku-nlp/deberta-v2-base-japanese")
model = AutoModel.from_pretrained("ku-nlp/deberta-v2-base-japanese")

Some weights of the model checkpoint at ku-nlp/deberta-v2-base-japanese were not used when initializing DebertaV2Model: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:

###bert_tokenizer = BertTokenizer(vocab_file, do_lower_case=False, do_basic_tokenize=False)

In [11]:
from pyknp import Juman

class JumanTokenizer():
    def __init__(self):
        self.juman = Juman()

    def tokenize(self, text):
        result = self.juman.analysis(text)
        return [mrph.midasi for mrph in result.mrph_list()]

In [12]:
juman_tokenizer = JumanTokenizer()

In [13]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
    
def preprocessing_text(text):
       
        text = mojimoji.han_to_zen(text) 
        
        text = re.sub(r'[0-9 ０-９]+', '0', text)  

        text = re.sub('\r', '', text)
        text = re.sub('\n', '', text)

        text = text.replace("、", " ")
        text = text.replace("。", " ")
        
        for p in string.punctuation:
            if (p == ".") or (p == ","):
                continue
            else:
                text = text.replace(p, " ")
            return text

def tokenizer_with_preprocessing(text):
        text = preprocessing_text(text)
        tokens = juman_tokenizer.tokenize(text)
        tokens = tokenizer.tokenize(" ".join(tokens))
        ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens+ ["[SEP]"])
        return ids

def preprocessing_sentences(text):
        for i in range(len(text)):
            text[i] = preprocessing_text(text[i])
        return text

### 訓練データ

In [114]:
# 作成したデータの読み込み
df_train = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/train_data(5).csv", header=0)
#df_train = df_train.dropna(how='any') # nanのところは落とす（欠損値の削除）

In [115]:
# データの確認
print(f'データサイズ： {df_train.shape}')
display(df_train.sample(10))

データサイズ： (7000, 2)


,text,label
1909,ピューロランド 関係 ない けど 面白い 漫画 見つけ た よ !! 彼氏 、 旦那 の 愚...,0
4327,すべては夢見ることから始まる。\n\n【ウォルト・ディズニー】＜ＵＲＬ＞,0
6869,眠く て 、 洗面 所 ( 特に 入ら ない か なー と 「 テレビ を 操作 し て 着...,0
6139,ピューロランド落選していく？,0
3643,ひな ちと ユニバ だっ た ??? かわいい 赤ずきん を 引き連れ て 狼 し て まし...,0
2390,今年はサンリオピューロランドに行く？,1
5098,今月 は あと 2回 撮影 の 予定 が あっ て 、 来月 は たくさん ヾ ( ・ ω ...,0
1842,おととい の マツコの知らない世界 見 忘れ た から GYAO で 見直せ ば いい か ...,0
2222,なんとディズニーデラックスでは月額７００円（初月無料）でアイアンマン２？ＩＷまで（ホームカミ...,0
3811,みんな で ユニバ き て ほしい な ?,0


### テストデータ

In [116]:
# 作成したデータの読み込み
df_test = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/test_data(5).csv",header=0)

In [117]:
print(f'データサイズ： {df_test.shape}')
display(df_test.sample(10))

データサイズ： (1998, 2)


,text,label
1227,1104 ホラナイピエロパタパタ くん と オレンテ くん が ジャンプ し て 遊ん でる...,0
339,モンスターズインク面白かった、明日ユニバーシティみるー,0
1512,続き 日本 の ハロウィン どうにか なら ない の か な ... 津野 さん の アカウ...,1
352,ディズニーキャラ以外で死ぬまでにしたい（着たい）コスプレ\n・メイド（キャンディフルーツ）\...,0
736,大阪 の 即売会 に 参加 し て 1日 早く 入っ て 同人 仲間 と ユニバ で 遊ん ...,0
1699,イベント 主催 者 から コンラッド NY の ツイン を 無料 で 1部 屋 与え られ ...,0
1175,ダウンロード 遅 すぎ て 生き てけない から 今日 は 寝よ か な 。 うん 明日 ユ...,0
1221,今更 ながら ユニバイベー なみ さん に やっ っと 会え た ! あ きは に ひっ さ...,1
1505,「 私たち は 前進 を 続け 、 新しい 扉 を 開き 、 新た な こと を なし 遂げ...,0
1155,11 . 79 修学旅行 ・ 広島 … 写真 は デジカメ で ? ホテル 綺麗 だっ た ...,0


### valデータ

In [118]:
# 作成したデータの読み込み
df_val = pd.read_csv("/home/sakulab/workspace/yaguchi/research/Master/data/意見データ(5分割交差検証用)/val_data(5).csv",header=0)

In [119]:
print(f'データサイズ： {df_val.shape}')
display(df_val.sample(10))

データサイズ： (996, 2)


,text,label
319,前 お 店 で ディズニー の バッグ 持っ て 精算 待ち し て たら 、 店 の 女の...,1
510,【 AM 新 景品 情報 】 # ディズニー の # ダンボ より 寝そべり # ぬいぐるみ...,1
788,スパイダーマンザ・ライド の キュー ライン 2 F は ヒーローヴィラン の 説明 欄 が...,0
538,がんばっ て 今 まで の お つかれ ディズニー し て こよ ?,0
635,撮影 納め し たく ない けど 、 今日 ダンス を 撮影 しよ う と 思っ て 調べ ...,0
285,ディズニー の キャラ が 苦手 っていう 致命 的 な 欠点 が ある,0
583,特別支援教育 って 本質的 に は 個人 の 学び の 保障 な はず な のに 、 今 な...,0
102,ピューロランド楽しんできました！\n\n私の大好きなビリーブ衣装のキティちゃんとダニエルくん...,1
30,リッツカールトンやシェラトンでＶＩＰサービスを受けることができる「ＳＰＧアメックスカード」\...,0
220,鉾田の詐欺ハウステンボスわろた,0


In [120]:
# モデルに飲ませるデータと、ラベルを準備
text_train = df_train.text.values
labels_train = df_train.label.values


## 確認
print(' Original: ', text_train[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_train[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_train[0]))

 Original:  え 、 ピューロランド 行き たい 。 。 。 誰か 連れ て っ て ← 絶対 SKY ー HI 可愛い やん 。 いや 、 カッコイイ パターン も あり ?
Tokenized:  ['え', '\u3000', '\u3000', '\u3000', 'ピュー', 'ロランド', '\u3000', '行き', '\u3000', 'たい', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '\u3000', '誰', 'か', '\u3000', '連れ', '\u3000', 'て', '\u3000っ', '\u3000', 'て', '\u3000', '←', '\u3000', '絶対', '\u3000', 'ＳＫＹ', '\u3000', 'ー', '\u3000', 'ＨＩ', '\u3000', '可愛い', '\u3000', 'やん', '\u3000', '\u3000', '\u3000', 'いや', '\u3000', '\u3000', '\u3000', 'カッコイイ', '\u3000', 'パターン', '\u3000', 'も', '\u3000', 'あり', '\u3000', '？']
Token IDs:  [1, 2654, 334, 334, 334, 4255, 1552, 2459, 334, 1459, 334, 371, 334, 334, 334, 334, 334, 334, 334, 1122, 294, 334, 6629, 334, 1872, 334, 1138, 334, 1872, 334, 16433, 334, 2372, 334, 475, 17490, 334, 2994, 334, 270, 14846, 334, 4820, 334, 17171, 334, 334, 334, 5792, 334, 334, 334, 19456, 895, 895, 334, 4333, 334, 276, 334, 317, 334, 354, 2]


In [121]:
# モデルに飲ませるデータと、ラベルを準備
text_val = df_val.text.values
labels_val = df_val.label.values

## 確認
print(' Original: ', text_val[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_val[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_val[0]))

 Original:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですね
なるほどこの世界はなろう小説でしたか
Tokenized:  ['そう', 'いえば', '大阪', 'の', 'ウンタラ', 'で', 'は', 'ＵＳＪ', 'の', 'ウンタラ', 'を', 'ウンタラ', 'する', 'そうです', 'ね', 'なるほど', 'この', '世界', 'は', 'なろう', '小説', 'でした', 'か']
Token IDs:  [1, 849, 3551, 648, 262, 13561, 16889, 269, 266, 7970, 1667, 262, 13561, 16889, 265, 13561, 16889, 279, 1252, 383, 17910, 341, 427, 266, 11473, 1603, 588, 294, 2]


In [122]:
# モデルに飲ませるデータと、ラベルを準備
text_test = df_test.text.values
labels_test = df_test.label.values

## 確認
print(' Original: ', text_test[0])
print('Tokenized: ', juman_tokenizer.tokenize(preprocessing_text(text_test[0])))
print('Token IDs: ', tokenizer_with_preprocessing(text_test[0]))

 Original:  レゴランドの向いにホテルが立ってるんですよー
私も今日、初めて行きました；；
レストランはビュッフェタイプでした。
他にもバーとかありましたよ？
少々お高いですが、わりと楽しめました！
Tokenized:  ['レゴ', 'ランド', 'の', '向い', 'に', 'ホテル', 'が', '立って', 'る', 'んです', 'よー', '私', 'も', '今日', '\u3000', '初めて', '行き', 'ました', '；', '；', 'レストラン', 'は', 'ビュッフェ', 'タイプ', 'でした', '\u3000', '他', 'に', 'も', 'バー', 'と', 'か', 'あり', 'ました', 'よ', '？', '少々', 'お', '高い', 'です', 'が', '\u3000', 'わりと', '楽しめ', 'ました', '！']
Token IDs:  [1, 1409, 1120, 8416, 262, 6632, 457, 264, 1335, 267, 6008, 587, 602, 17697, 426, 276, 1464, 334, 964, 1459, 307, 377, 377, 3672, 266, 270, 11804, 1300, 3043, 1268, 588, 334, 477, 264, 276, 1624, 268, 294, 317, 307, 423, 354, 12233, 295, 672, 302, 267, 334, 28643, 7742, 307, 308, 2]


In [123]:
text_train = preprocessing_sentences(text_train)
text_val = preprocessing_sentences(text_val)
text_test = preprocessing_sentences(text_test)
print('Preprocessing: ', text_train[0])
print('Preprocessing: ', text_val[0])
print('Preprocessing: ', text_test[0])

Preprocessing:  え　 　ピューロランド　行き　たい　 　 　 　誰か　連れ　て　っ　て　←　絶対　ＳＫＹ　ー　ＨＩ　可愛い　やん　 　いや　 　カッコイイ　パターン　も　あり　？
Preprocessing:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですねなるほどこの世界はなろう小説でしたか
Preprocessing:  レゴランドの向いにホテルが立ってるんですよー私も今日 初めて行きました；；レストランはビュッフェタイプでした 他にもバーとかありましたよ？少々お高いですが わりと楽しめました！


### IDへ変換

In [124]:
input_ids_train = []
attention_masks_train = []
max_len = 128

# 1文づつ処理
for sentence in text_train:
    encoded_dict_train = tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_train.append(encoded_dict_train['input_ids'])
    attention_masks_train.append(encoded_dict_train['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)

# tensor型に変換
labels_train = torch.tensor(labels_train)

# 確認
print('Original: ', text_train[0])
print('Token IDs:', input_ids_train[0])

/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  え　 　ピューロランド　行き　たい　 　 　 　誰か　連れ　て　っ　て　←　絶対　ＳＫＹ　ー　ＨＩ　可愛い　やん　 　いや　 　カッコイイ　パターン　も　あり　？
Token IDs: tensor([    1,  2654, 31958,   334, 10681,   555,   702,  2459, 31958, 20476,
        31958,  5981, 31958,   334,   334,   334,  6953,   644, 31958,  4018,
          920, 31958,   359, 31958,  1138, 31958,   359, 31958, 26637, 31958,
        21631, 31958,   800, 17490, 31958,   555, 31958, 14846, 31958, 29968,
        31958,  1159,   956, 31958,   334,  7443, 31958,   334,   695,  7532,
          895,   895, 31958,  1372, 17715, 31958,  1512, 31958, 14671, 31958,
        11897,     2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   

In [125]:
input_ids_val = []
attention_masks_val = []
max_len = 128


# 1文づつ処理
for sentence in text_val:
    encoded_dict_val = tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_val.append(encoded_dict_val['input_ids'])
    attention_masks_val.append(encoded_dict_val['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val = torch.cat(attention_masks_val, dim=0)

# tensor型に変換
labels_val = torch.tensor(labels_val)

# 確認
print('Original: ', text_val[0])
print('Token IDs:', input_ids_val[0])

Original:  そういえば大阪のウンタラではＵＳＪのウンタラをウンタラするそうですねなるほどこの世界はなろう小説でしたか
Token IDs: tensor([    1,   849,   457, 11628,  4092,   419,  8152, 16889,  5405,  6053,
         1667,   419,  8152, 16889,  6731,  8152, 16889,  2648,  1416,   993,
         2144, 28083,   362,  3260, 15511,  2402, 15735,  2975,   644,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,    

In [126]:
input_ids_test = []
attention_masks_test = []
max_len = 128

# 1文づつ処理
for sentence in text_test:
    encoded_dict_test = tokenizer.encode_plus( # input_ids, token_type_ids, attention_mask を出力する
                        sentence,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = max_len + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    input_ids_test.append(encoded_dict_test['input_ids'])
    attention_masks_test.append(encoded_dict_test['attention_mask']) # 実質的なトークンがある部分のみ１になる

#input_ids_train.append(sentences_train['input_ids'])
#attention_masks_train.append(sentences_train['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)

# tensor型に変換
labels_test = torch.tensor(labels_test)

# 確認
print('Original: ', text_test[0])
print('Token IDs:', input_ids_test[0])

Original:  レゴランドの向いにホテルが立ってるんですよー私も今日 初めて行きました；；レストランはビュッフェタイプでした 他にもバーとかありましたよ？少々お高いですが わりと楽しめました！
Token IDs: tensor([    1,  1409,  1120,  2459,   419,  5285,   457,   370, 11826,  1202,
        15045,   369,  1366,   677,  2472,   555,   771,  1512,  2283,   964,
        20476,   645,  1260,  9278,  9278, 29829,  1428, 11804,  1300,  3043,
        21581,  2975,   477, 11854,  1661,   515,   644, 14671,   645,  1260,
         2472, 11897,  8868,  1113,   552, 15053,  7218, 28643,  2859,   494,
         1077,   645,  1260, 10772,     2,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

### データローダーの準備

In [127]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train) # input_ids_train, attention_masks_train, labels_train を使ってデータセットを作成
val_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# データローダーの作成
batch_size = 16

# datsets = [データセット全て]
# Dataloader = [[batch_1], [batch_2], ... [batch_n]]

# 訓練データローダー
train_dataloader = DataLoader( # Dataloader : データセットからデータをバッチサイズに固めて返すモジュール
    train_dataset,  
    sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化（sampler : datasetsのバッチの固め方を決める事のできる設定）
    batch_size = batch_size
)

# 検証データローダー
validation_dataloader = DataLoader(
    val_dataset, 
    sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
    batch_size = batch_size
)

# テストデータローダー
test_dataloader = DataLoader(
    test_dataset, 
    sampler = SequentialSampler(test_dataset), # 順番にデータを取得してバッチ化
    batch_size = batch_size
)

In [128]:
from transformers import DebertaV2Model

class DebertaV2Classifier(nn.Module):
  def __init__(self, model_name):
    super(DebertaV2Classifier, self).__init__()
    
    # DeBERTa V2モデルをロード
    self.deberta = DebertaV2Model.from_pretrained(model_name)
    
    # DeBERTaの隠れ層の次元数は768, カテゴリ数が2
    self.linear = nn.Linear(768, 2)
    
    # 重み初期化処理
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)

  def forward(self, input_ids):
    # last_hidden_stateを受け取る
    outputs = self.deberta(input_ids)
    
    # 先頭トークンclsのベクトルだけ取得
    vec = outputs.last_hidden_state[:,0,:]
    vec = vec.view(-1, 768)
    
    # 全結合層でクラス分類用に次元を変換
    out = self.linear(vec)
    
    return out

model_name = "ku-nlp/deberta-v2-base-japanese"
model = DebertaV2Classifier(model_name)

Some weights of the model checkpoint at ku-nlp/deberta-v2-base-japanese were not used when initializing DebertaV2Model: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [129]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### ファインチューニング

In [130]:
# ファインチューニング
for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.deberta.encoder.layer[-1].named_parameters():
    param.requires_grad = True

for name, param in model.linear.named_parameters():
    param.requires_grad = True

# 最適化関数の設定
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()


/home/sakulab/anaconda3/envs/[yaguchi]/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [131]:
EarlyStopping_path = '/home/sakulab/workspace/yaguchi/research/Master/code/LLM/Deberta/(ku-nlp)results/val5(epoch10)_chp'

class EarlyStopping:
    """earlystoppingクラス"""

    def __init__(self, patience=10, verbose=False, path= EarlyStopping_path ):
        """引数：最小値の非更新数カウンタ、表示設定、モデル格納path"""

        self.patience = patience    #設定ストップカウンタ
        self.verbose = verbose      #表示の有無
        self.counter = 0            #現在のカウンタ値
        self.best_score = None      #ベストスコア
        self.early_stop = False     #ストップフラグ
        self.val_loss_min = np.Inf   #前回のベストスコア記憶用
        self.path = path             #ベストモデル格納path

    def __call__(self, val_loss, model):
        """
        特殊(call)メソッド
        実際に学習ループ内で最小lossを更新したか否かを計算させる部分
        """
        score = -val_loss

        if self.best_score is None:  #1Epoch目の処理
            self.best_score = score   #1Epoch目はそのままベストスコアとして記録する
            self.checkpoint(val_loss, model)  #記録後にモデルを保存してスコア表示する
        elif score < self.best_score:  # ベストスコアを更新できなかった場合
            self.counter += 1   #ストップカウンタを+1
            if self.verbose:  #表示を有効にした場合は経過を表示
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')  #現在のカウンタを表示する 
            if self.counter >= self.patience:  #設定カウントを上回ったらストップフラグをTrueに変更
                self.early_stop = True
        else:  #ベストスコアを更新した場合
            self.best_score = score  #ベストスコアを上書き
            self.checkpoint(val_loss, model)  #モデルを保存してスコア表示
            self.counter = 0  #ストップカウンタリセット

    def checkpoint(self, val_loss, model):
        '''ベストスコア更新時に実行されるチェックポイント関数'''
        if self.verbose:  #表示を有効にした場合は、前回のベストスコアからどれだけ更新したか？を表示
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  #ベストモデルを指定したpathに保存
        self.val_loss_min = val_loss  #その時のlossを記録する

In [132]:
# 学習結果の保存用
history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc':[]
    }

def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):
    #★EarlyStoppingクラスのインスタンス化とエアearylistoppingの設定
    earlystopping = EarlyStopping(patience=10)

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    

    # ネットワークをGPUへ
    model.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size
    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに
               
            else:
                model.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                #inputs = batch.Text[0].to(device)  # 文章 input_ids
                inputs = batch[0].to(device)  # 文章 input_ids
                #labels = batch.Label.to(device)  # ラベル
                labels = batch[2].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # Bertに入力
                    outputs= model(inputs)
                    
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                       
                      
                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)
      
            # epochごとのlossと正解率
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

            
            if phase == 'train':
                 history['train_loss'].append(epoch_loss)
                 history['train_acc'].append(epoch_acc)
            else:
                history['val_loss'].append(epoch_loss)   
                history['val_acc'].append(epoch_acc)
                
        earlystopping(epoch_loss, model) #callメソッド呼び出し
        if earlystopping.early_stop: #ストップフラグがTrueの場合、breakでforループを抜ける
              print("Early Stopping!")
              break

    return model

In [133]:
num_epochs = 10
dataloaders_dict = {"train": train_dataloader, "val": validation_dataloader}
model_trained = train_model(model, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
Epoch 1/10 | train |  Loss: 0.5150 Acc: 0.7553
Epoch 1/10 |  val  |  Loss: 0.5189 Acc: 0.7239
Epoch 2/10 | train |  Loss: 0.4987 Acc: 0.7577
Epoch 2/10 |  val  |  Loss: 0.5108 Acc: 0.7540
Epoch 3/10 | train |  Loss: 0.4896 Acc: 0.7591
Epoch 3/10 |  val  |  Loss: 0.5244 Acc: 0.7199
Epoch 4/10 | train |  Loss: 0.4840 Acc: 0.7624
Epoch 4/10 |  val  |  Loss: 0.5069 Acc: 0.7470
Epoch 5/10 | train |  Loss: 0.4768 Acc: 0.7666
Epoch 5/10 |  val  |  Loss: 0.4997 Acc: 0.7520
Epoch 6/10 | train |  Loss: 0.4740 Acc: 0.7680
Epoch 6/10 |  val  |  Loss: 0.5066 Acc: 0.7359
Epoch 7/10 | train |  Loss: 0.4666 Acc: 0.7731
Epoch 7/10 |  val  |  Loss: 0.5024 Acc: 0.7530
Epoch 8/10 | train |  Loss: 0.4629 Acc: 0.7743
Epoch 8/10 |  val  |  Loss: 0.4980 Acc: 0.7550
Epoch 9/10 | train |  Loss: 0.4524 Acc: 0.7844
Epoch 9/10 |  val  |  Loss: 0.4963 Acc: 0.7480
Epoch 10/10 | train |  Loss: 0.4410 Acc: 0.7893
Epoch 10/10 |  val  |  Loss: 0.4996 Acc: 0.7560


In [134]:
# ベストモデルを適応させる
model_trained.load_state_dict(torch.load(EarlyStopping_path))

<All keys matched successfully>

### 正解率の算出

In [135]:
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_trained.eval()   # モデルを検証モードに
model_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

y_pred = []

for batch in tqdm(test_dataloader):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #inputs = batch.Text[0].to(device)  # 文章
    inputs = batch[0].to(device)  # 文章
    #labels = batch.Label.to(device)  # ラベル
    labels = batch[2].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

       
        outputs = model_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        if y_pred == []:
          y_pred = preds
        else:
          y_pred = torch.cat([y_pred, preds], dim=0)
        
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dataloader.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dataloader.dataset), epoch_acc))

100%|██████████| 125/125 [00:05<00:00, 23.69it/s]

テストデータ1998個での正解率：0.7693


In [136]:
y_pred = y_pred.to('cpu')

### 分類評価指標4つでの結果

In [137]:
# 評価指標の算出
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# 評価指標算出メソッドの作成
def print_metrics_by_cutoff(y, y_pred, model, cutoff):
    """
    指定した閾値に基づくAccuracy, Precision, Recall, F1-scoreを出力する
    """
    #y_pred = model.predict_proba(X)[:, 1] >= cutoff
    print('accuracy = {:.4f}'.format(accuracy_score(y_true=y, y_pred=y_pred)))
    print('precision = {:.4f}'.format(precision_score(y_true=y, y_pred=y_pred)))
    print('recall = {:.4f}'.format(recall_score(y_true=y, y_pred=y_pred)))
    print('f1 score = {:.4f}'.format(f1_score(y_true=y, y_pred=y_pred)))
    # Accuracy, Precision, Recall, F1-score
print_metrics_by_cutoff(y=labels_test, y_pred=y_pred, model=model_trained, cutoff=0.5)

accuracy = 0.7693
precision = 0.5566
recall = 0.2433
f1 score = 0.3386


In [1]:
#GPUのメモリ解放
! kill -9 $(lsof -t /dev/nvidia*)

In [138]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Jul 31 12:44:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:81:00.0  On |                  N/A |
| 42